<a href="https://colab.research.google.com/github/superishi/Cybersecurity-Flow-Anomaly-Detection/blob/main/DataPreprocessing.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

#Loading and Preparing Data

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
origData = pd.read_csv("/content/drive/MyDrive/DATA3001 Cyber/datascience-cis14d.csv", header = 0)
origData.head()

,time,FromInternetTCPPort443IPurn:ietf:params:mud:controllerByteCount,FromInternetTCPPort443IPurn:ietf:params:mud:controllerPacketCount,FromInternetTCPPort554IPurn:ietf:params:mud:controllerByteCount,FromInternetTCPPort554IPurn:ietf:params:mud:controllerPacketCount,FromInternetTCPPort80IPurn:ietf:params:mud:controllerByteCount,FromInternetTCPPort80IPurn:ietf:params:mud:controllerPacketCount,FromInternetUDPPort67IPurn:ietf:params:mud:controllerByteCount,FromInternetUDPPort67IPurn:ietf:params:mud:controllerPacketCount,ToInternet0x0800PortAllIPurn:ietf:params:mud:controllerByteCount,ToInternet0x0800PortAllIPurn:ietf:params:mud:controllerPacketCount,ToInternetTCPPort443IPurn:ietf:params:mud:controllerByteCount,ToInternetTCPPort443IPurn:ietf:params:mud:controllerPacketCount,ToInternetTCPPort554IPurn:ietf:params:mud:controllerByteCount,ToInternetTCPPort554IPurn:ietf:params:mud:controllerPacketCount,ToInternetTCPPort80IPurn:ietf:params:mud:controllerByteCount,ToInternetTCPPort80IPurn:ietf:params:mud:controllerPacketCount,ToInternetUDPPort1024IPurn:ietf:params:mud:controllerByteCount,ToInternetUDPPort1024IPurn:ietf:params:mud:controllerPacketCount,ToLocalUDPPort5353IP224.0.0.251/32ByteCount,ToLocalUDPPort5353IP224.0.0.251/32PacketCount,ToLocalUDPPort67IP255.255.255.255/32ByteCount,ToLocalUDPPort67IP255.255.255.255/32PacketCount,device_mac
0,1629733373023000000,0,0,0,0,0,0,0,0,1784,4,0,0,52130144,38758,0,0,0,0,0,0,0,0,21:fa:27:56
1,1629733373023000000,0,0,0,0,0,0,0,0,14648,52,0,0,31852,206,353234,834,50901698,35662,0,0,0,0,bd:1c:cf:1a
2,1629733373036000000,0,0,0,0,0,0,0,0,14648,52,0,0,34180,242,241648,558,54056558,39026,0,0,1812,4,bd:0d:a9:e4
3,1629733373036000000,0,0,0,0,0,0,0,0,7136,16,0,0,5184,24,0,0,29017738,21050,0,0,0,0,bd:0d:af:0b
4,1629733373036000000,0,0,0,0,0,0,2221800,6348,3568,8,0,0,5184,24,196414,460,0,0,0,0,0,0,ee:dd:38:57


In [ ]:
origData

,time,FromInternetTCPPort443IPurn:ietf:params:mud:controllerByteCount,FromInternetTCPPort443IPurn:ietf:params:mud:controllerPacketCount,FromInternetTCPPort554IPurn:ietf:params:mud:controllerByteCount,FromInternetTCPPort554IPurn:ietf:params:mud:controllerPacketCount,FromInternetTCPPort80IPurn:ietf:params:mud:controllerByteCount,FromInternetTCPPort80IPurn:ietf:params:mud:controllerPacketCount,FromInternetUDPPort67IPurn:ietf:params:mud:controllerByteCount,FromInternetUDPPort67IPurn:ietf:params:mud:controllerPacketCount,ToInternet0x0800PortAllIPurn:ietf:params:mud:controllerByteCount,ToInternet0x0800PortAllIPurn:ietf:params:mud:controllerPacketCount,ToInternetTCPPort443IPurn:ietf:params:mud:controllerByteCount,ToInternetTCPPort443IPurn:ietf:params:mud:controllerPacketCount,ToInternetTCPPort554IPurn:ietf:params:mud:controllerByteCount,ToInternetTCPPort554IPurn:ietf:params:mud:controllerPacketCount,ToInternetTCPPort80IPurn:ietf:params:mud:controllerByteCount,ToInternetTCPPort80IPurn:ietf:params:mud:controllerPacketCount,ToInternetUDPPort1024IPurn:ietf:params:mud:controllerByteCount,ToInternetUDPPort1024IPurn:ietf:params:mud:controllerPacketCount,ToLocalUDPPort5353IP224.0.0.251/32ByteCount,ToLocalUDPPort5353IP224.0.0.251/32PacketCount,ToLocalUDPPort67IP255.255.255.255/32ByteCount,ToLocalUDPPort67IP255.255.255.255/32PacketCount,device_mac
0,1629733373023000000,0,0,0,0,0,0,0,0,1784,4,0,0,52130144,38758,0,0,0,0,0,0,0,0,21:fa:27:56
1,1629733373023000000,0,0,0,0,0,0,0,0,14648,52,0,0,31852,206,353234,834,50901698,35662,0,0,0,0,bd:1c:cf:1a
2,1629733373036000000,0,0,0,0,0,0,0,0,14648,52,0,0,34180,242,241648,558,54056558,39026,0,0,1812,4,bd:0d:a9:e4
3,1629733373036000000,0,0,0,0,0,0,0,0,7136,16,0,0,5184,24,0,0,29017738,21050,0,0,0,0,bd:0d:af:0b
4,1629733373036000000,0,0,0,0,0,0,2221800,6348,3568,8,0,0,5184,24,196414,460,0,0,0,0,0,0,ee:dd:38:57
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12853527,1630942945712000000,0,0,0,0,0,0,0,0,1784,4,0,0,58635900,44818,227530,520,0,0,0,0,0,0,bd:26:0c:31
12853528,1630942945712000000,0,0,0,0,0,0,0,0,3568,8,0,0,58842848,44892,137278,300,0,0,0,0,0,0,bd:26:0c:2f
12853529,1630942945713000000,0,0,0,0,0,0,0,0,3748,10,0,0,58103808,44692,202004,472,0,0,0,0,0,0,bd:26:0c:44
12853530,1630942945713000000,0,0,0,0,0,0,0,0,3568,8,0,0,58824052,45028,202480,472,0,0,0,0,0,0,bd:26:0c:30


In [ ]:
origDataNp = origData.to_numpy()

Extracting Device ID and Time from rest of data

In [ ]:
deviceKey = np.expand_dims(origDataNp[:,-1], axis = 1)
timeKey = np.expand_dims(origDataNp[:,0], axis = 1)


dataPrimaryKey = np.concatenate((deviceKey, timeKey), axis = 1)
dataPrimaryKey

array([['21:fa:27:56', 1629733373023000000],
       ['bd:1c:cf:1a', 1629733373023000000],
       ['bd:0d:a9:e4', 1629733373036000000],
       ...,
       ['bd:26:0c:44', 1630942945713000000],
       ['bd:26:0c:30', 1630942945713000000],
       ['bd:24:bb:50', 1630942945713000000]], dtype=object)

In [ ]:
flowData = origDataNp[:,1:-1]
flowData[4]

array([0, 0, 0, 0, 0, 0, 2221800, 6348, 3568, 8, 0, 0, 5184, 24, 196414,
       460, 0, 0, 0, 0, 0, 0], dtype=object)

# Normalisation and Dimensionality Reduction

In [ ]:
from sklearn import preprocessing

In [ ]:
normalized_flowData = preprocessing.normalize(flowData)
normalized_flowData

NameError: ignored

In [ ]:
from sklearn.decomposition import TruncatedSVD # TruncatedSVD purportedly works better with sparse data than PCA

Not enough RAM to fit SVD on whole data, subset of the data taken using random sampling 

In [ ]:
no_of_samples = 500000
rows_to_sample = np.random.choice(normalized_flowData.shape[0], no_of_samples, replace=False)
sample_normalized_flowData = normalized_flowData[rows_to_sample]
sample_normalized_flowData.shape

(500000, 22)

In [ ]:
def svd_explained_variance(no_of_features):
  svd = TruncatedSVD(no_of_features)
  svd.fit(sample_normalized_flowData)
  print(no_of_features, ": " ,sum(svd.explained_variance_ratio_))

In [ ]:
for i in range(0,22):
  svd_explained_variance(i)

In [ ]:
svd = TruncatedSVD(4)
svd.fit(sample_normalized_flowData)
dimReduced_flow_Data = svd.transform(normalized_flowData)

In [ ]:
print(dimReduced_flow_Data.shape)
print(svd.explained_variance_)
print(sum(svd.explained_variance_ratio_))
dimReduced_flow_Data

(12853532, 4)
[0.23311941 0.14940142 0.19929638 0.05770115]
0.9474114957229653


array([[ 1.78974505e-03,  9.33792222e-01, -2.92533432e-01,
        -2.06008143e-01],
       [ 9.99957685e-01, -1.18795058e-03,  4.05672254e-04,
        -2.47240480e-04],
       [ 9.99954826e-01, -1.88815591e-03, -1.95721610e-03,
        -9.70155956e-05],
       ...,
       [ 1.81361812e-03,  9.34783411e-01, -2.89207270e-01,
        -2.06209129e-01],
       [ 1.81337868e-03,  9.34772869e-01, -2.89240281e-01,
        -2.06210602e-01],
       [ 9.99956972e-01, -1.68555319e-03,  3.81737620e-04,
        -2.98133143e-04]])

# Transforming time format and preparing time series data

In [ ]:
from datetime import datetime

In [ ]:
def epoch_to_datetime(epoch_time):
    dt = datetime.fromtimestamp(epoch_time // 1000000000)
    #date_and_time = dt.strftime('%Y-%m-%d %H:%M:%S')
    timecode = dt.strftime('%Y%m%d%H%M%S')  
    return timecode

In [ ]:
#print(epoch_to_datetime(dataPrimaryKey[0,1]).shape)
print(epoch_to_datetime(dataPrimaryKey[1,1]))
print(epoch_to_datetime(dataPrimaryKey[-1,1]))
#decoded_time currently placeholder array
decoded_time = dataPrimaryKey[:,1]
decoded_time.shape

20210823154253
20210906154225


(12853532,)

In [ ]:
# try changing so that array is updated in real time
# also separate epoch_to_datetime into two functions
i = 0
for epoch_time in dataPrimaryKey[:,1]:
    decoded_time[i] = epoch_to_datetime(epoch_time) 
    i += 1
    if (i % 10000) == 0:
        print(i)

10000
20000
30000
40000
50000
60000
70000
80000
90000
100000
110000
120000
130000
140000
150000
160000
170000
180000
190000
200000
210000
220000
230000
240000
250000
260000
270000
280000
290000
300000
310000
320000
330000
340000
350000
360000
370000
380000
390000
400000
410000
420000
430000
440000
450000
460000
470000
480000
490000
500000
510000
520000
530000
540000
550000
560000
570000
580000
590000
600000
610000
620000
630000
640000
650000
660000
670000
680000
690000
700000
710000
720000
730000
740000
750000
760000
770000
780000
790000
800000
810000
820000
830000
840000
850000
860000
870000
880000
890000
900000
910000
920000
930000
940000
950000
960000
970000
980000
990000
1000000
1010000
1020000
1030000
1040000
1050000
1060000
1070000
1080000
1090000
1100000
1110000
1120000
1130000
1140000
1150000
1160000
1170000
1180000
1190000
1200000
1210000
1220000
1230000
1240000
1250000
1260000
1270000
1280000
1290000
1300000
1310000
1320000
1330000
1340000
1350000
1360000
1370000
1380000
1390

In [ ]:
decoded_time.shape

(12853532,)

In [ ]:
decoded_time = np.expand_dims(decoded_time, axis = 1)

In [ ]:
print("After: ", decoded_time.shape)
print(decoded_time)

After:  (12853532, 1)
[['20210823154253']
 ['20210823154253']
 ['20210823154253']
 ...
 ['20210906154225']
 ['20210906154225']
 ['20210906154225']]


In [ ]:
deviceID = dataPrimaryKey[:,0]
deviceID = np.expand_dims(deviceID, axis = 1)
print(deviceID.shape)
deviceID

(12853532, 1)


array([['21:fa:27:56'],
       ['bd:1c:cf:1a'],
       ['bd:0d:a9:e4'],
       ...,
       ['bd:26:0c:44'],
       ['bd:26:0c:30'],
       ['bd:24:bb:50']], dtype=object)

In [ ]:
final_dataKey = np.concatenate((deviceID, decoded_time), axis = 1)
final_dataKey

array([['21:fa:27:56', '20210823154253'],
       ['bd:1c:cf:1a', '20210823154253'],
       ['bd:0d:a9:e4', '20210823154253'],
       ...,
       ['bd:26:0c:44', '20210906154225'],
       ['bd:26:0c:30', '20210906154225'],
       ['bd:24:bb:50', '20210906154225']], dtype=object)

In [ ]:
#print(dimReduced_flow_Data.shape)
#rows, cols = dimReduced_flow_Data.shape
#print(rows, cols)
final_data = np.ones((12853532, 6))
print(final_data.shape)

In [ ]:
for i in range(12853532):
 # final_data[i, 0] = final_dataKey[i,0]
  final_data[i, 1] = final_dataKey[i,1]
  final_data[i, 2] = dimReduced_flow_Data[i,0]
  final_data[i, 3] = dimReduced_flow_Data[i,1]
  final_data[i, 4] = dimReduced_flow_Data[i,2]
  final_data[i, 5] = dimReduced_flow_Data[i,3]
  if i % 10000 == 0:
    print(i)

In [ ]:
final_data[1,:]

array([ 1.00000000e+00,  2.02108232e+13,  9.99958746e-01, -1.14992719e-03,
        4.33052991e-04, -2.46243159e-04])

In [ ]:
#np.savetxt("data_key.csv", final_dataKey, delimiter=",")

ValueError: ignored

In [ ]:
df = pd.DataFrame(final_dataKey)
df.to_csv('/content/drive/MyDrive/DATA3001 Cyber/dataKey.csv',index=False)

In [ ]:
df2 = pd.DataFrame(dimReduced_flow_Data)
df2.to_csv('/content/drive/MyDrive/DATA3001 Cyber/flowData.csv',index=False)

# Wisesa's amazing aggregation solutions


In [ ]:
#splits a dataset into days and saves each of them
#returns the actual dates we've split it into
def daySplit(fileLoc):
  theData = pd.read_csv(fileLoc)
  theData["time"] = pd.to_datetime(theData["time"])
  theData.set_index("time", inplace=True)
  dates = []
  for i, group in theData.groupby(theData.index.date):
    print(f"Saving day {i}")
    group.to_csv(f"{i}{fileLoc}")
    dates.append(i)
  return dates
#reads data, aggregates it by time and device, pads it and saves it
def aggByTimeAndSave(file, timeStep = "h", outfile = None):
  if outfile == None:
    outfile = "agg " + timeStep + " " + file
  def fillDates(series, freq):
    series.reset_index(level="device_mac",drop=True,inplace=True)
    series = series.asfreq(freq=freq,fill_value=0)
    return series
  data = pd.read_csv(file)
  data["time"] = pd.to_datetime(data["time"])
  data = data.groupby(["device_mac",
                       pd.Grouper(key="time", freq=timeStep)]).sum()
  #pad unobserved with 0s makes the whole thing take 30 secs longer
  data = data.groupby("device_mac").apply(lambda s: fillDates(s, timeStep))
  data.to_csv(outfile)

#samples flowdata randomly from a datafile. Use with SVD
def sampleTimeBlock(file, n, dropKeys = False, replace = True):
  data = pd.read_csv(file).sample(n, replace = replace)
  if dropKeys:
    data.drop(["time", "device_mac"],axis = 1,  inplace = True)
  return data
#samples a device at random and all of its flow data
def sampleDeviceFlow(file, n, replace = True):
  data = pd.read_csv(file)
  data = data.groupby("device_mac").sample(n, replace = replace)
  return data.reset_index()

#can take in a filepath or dataset
#calls transform on the flowData and returns the resulting dataframe,
#optionally saves it into outfile
def applyFlowTransform(data, transform, outfile = None):
  if isinstance(data, str):
    data = pd.read_csv(data)
  flows = data.drop(["time", "device_mac"], axis = 1)
  data = data.filter(["time", "device_mac"], axis = 1)
  flows = transform(flows)
  flows = pd.DataFrame(flows, columns = 
                       [f"v{f}" for f in range(flows.shape[1])],
                       index = data.index)
  data = pd.concat([data, flows],join = "outer", axis=1)
  if outfile != None:
    data.to_csv(outfile, index = False)
  return data

#####Recommended Procedure

In [ ]:
from sklearn.preprocessing import normalize
from sklearn.decomposition import TruncatedSVD


In [ ]:
TIMESTEP = "15min"

#copies from google drive to runtime
!cp "/content/drive/MyDrive/2021/DATA3001 Cyber/datascience-cis14d.csv" "wholeData.csv"

#aggregates whole dataset per hour. could take a couple minutes
aggByTimeAndSave("wholeData.csv", timeStep = TIMESTEP, outfile = "hourly.csv")

#do some normalization
applyFlowTransform("hourly.csv", normalize, "normalizedhourly.csv")

#samples from the aggregatted data and trains the SVD
TRAININGSIZE = 100000
trainingData = sampleTimeBlock("normalizedhourly.csv", TRAININGSIZE)

trainingData = trainingData.drop(["time", "device_mac"], axis = 1)
theSVD = TruncatedSVD(4)
theSVD.fit(trainingData)

del trainingData #won't be using it, so get rid of it. Don't want to crash RAM



In [ ]:
#splits aggregates into dailys split
dates = daySplit("normalizedhourly.csv")
#then applies the dementiality reduction on them
for date in dates:
  applyFlowTransform(date + " normalizedhourly.csv", theSVD.transform,
                     outfile = date + " transformed.csv")
#sample from daily data
trainingData = np.array(
    [sampleDeviceFlow(date + " transformed.csv", TRAININGSIZE // 15) for date in dates],
    )
theClusterer.fit(trainingData)
del trainingData #don't forget to delete when done

Saving day 2021-08-23
Saving day 2021-08-24
Saving day 2021-08-25
Saving day 2021-08-26
Saving day 2021-08-27
Saving day 2021-08-28
Saving day 2021-08-29
Saving day 2021-08-30
Saving day 2021-08-31
Saving day 2021-09-01
Saving day 2021-09-02
Saving day 2021-09-03
Saving day 2021-09-04
Saving day 2021-09-05
Saving day 2021-09-06


TypeError: ignored

####Advice for RAM management
NEVER have two instances of the whole dataset (pandas or numpy) on at the same time.

If possible, try to put all processing and training happen in functions that take files as input and saves edits to files that way the datasets auto free when we're done with them.

If we have to store some data as a DataFrame or np.array, delete them as soon as we're done

Don't worry about saving the reduced dataset though, the code below should save the SVD itself onto the googleDrive. Obviously replace "theSVD" and "theCluster" with the variable names of the actual model

In [ ]:
#for yoinking the SVD Model so we can run it on other datasets
import pickle
with open("/content/drive/MyDrive/2021/DATA3001 Cyber/svdDailyModel.p", "wb") as f:
  pickle.dump(theSVD, f)


In [ ]:
filePath = ("/content/drive/MyDrive/2021/DATA3001 Cyber/" + 
            "by15mins.csv")
applyFlowTransform("normalizedhourly.csv", theSVD.transform, filePath)


,time,device_mac,v0,v1,v2,v3
0,2021-08-23 15:30:00,21:fa:27:56,0.001770,0.935927,-0.283863,-0.207611
1,2021-08-23 15:45:00,21:fa:27:56,0.001770,0.935927,-0.283863,-0.207609
2,2021-08-23 16:00:00,21:fa:27:56,0.001770,0.935927,-0.283863,-0.207609
3,2021-08-23 16:15:00,21:fa:27:56,0.001770,0.935927,-0.283863,-0.207609
4,2021-08-23 16:30:00,21:fa:27:56,0.001770,0.935927,-0.283863,-0.207609
...,...,...,...,...,...,...
1013956,2021-09-06 14:30:00,ee:dd:4f:cd,0.999949,-0.002637,-0.003167,0.000083
1013957,2021-09-06 14:45:00,ee:dd:4f:cd,0.999955,-0.001989,-0.000924,-0.000064
1013958,2021-09-06 15:00:00,ee:dd:4f:cd,0.999955,-0.001982,-0.000896,-0.000070
1013959,2021-09-06 15:15:00,ee:dd:4f:cd,0.999949,-0.002609,-0.003071,0.000076
